## Objetivo
O objetivo desse tutorial é apresentar técnicas para preparação de dados utilizando pyspark.

## PySpark
O PySpark é uma biblioteca de Spark para Python.

## Antes de continuar
O arquivo **/home/class/Desktop/hands-on/all_hadoop.sh** será utilizado para controlar os serviços do hadoop, hdfs e hive.
Para continuar execute o comando abaixo:
```bash
sh /home/class/Desktop/hands-on/all_hadoop.sh start
```

Ao finalizar, execute o comando:
```bash
sh /home/class/Desktop/hands-on/all_hadoop.sh stop
```

## Criando uma pasta, carregando arquivos e listando conteúdo no HDFS
O HDFS é o sistema de arquivos distribuído do ecossistema hadoop. Ele será o repositório para armazenamento dos arquivos que serão processados pelo Spark.

Primeiramente, vamos alterar algumas permissões do HDFS.
```bash
su hdfs

hdfs dfs -chmod 777 /user

exit
```

Agora fazemos o upload dos arquivos para o HFDS.
```bash
hdfs dfs -mkdir /user/files

# Lembre-se de alterar "sample" pelo nome do arquivo que será carregado
hdfs dfs -put sample.csv /user/files/sample.csv # Carregando arquivo para o HDFS
```

## Usando pyspark para trabalhar sobre os arquivos

O código a baixo é um exemplo de como trabalhar com arquivos do HDFS no pyspark.
```python
import pyspark

# Cria uma sessão spark
spark = pyspark.sql.SparkSession.builder.master("local").appName("class").getOrCreate()

# Lê o csv salvo no hdfs
df = spark.read.csv("hdfs://localhost/user/files/sample.csv", header=True)

# Imprime a cabeçalho arquivo
print(df)
DataFrame[nome: string, idade: string, profissao: string]
```

Vamos para alguns exemplos de tratamento desses dados:

#### Mudando tipo de dado de uma coluna

Perceba que a coluna "idade" esta com tipo "string". O seguinte comando permite alterar o tipo da coluna.

```python
df = df.withColumn("idade", df["idade"].cast("int"))
```
*Esse mesmo comando pode ser utilizado para criar uma coluna, apenas mude o primeiro parâmetro*

#### Estatísticas de colunas numéricas

O seguinte comando pode ser usado para gerar estatísticas da base. Nesse caso, ele irá retornar informações sobre todos as colunas numéricas. Isso pode ajudar a avaliar alguns problemas, por exemplo, o valor mínimo "1" para idade.

```python
df.describe().show()
+-------+------------------+
|summary|             idade|
+-------+------------------+
|  count|                 6|
|   mean|29.166666666666668|
| stddev|15.904925861715503|
|    min|                 1|
|    max|                43|
+-------+------------------+
```

#### Removendo registros duplicados

O comando dropDuplicates, remove linhas duplicadas do dataframe.

```python
# Removendo duplicados considerando o match em todas as colunas.
df.dropDuplicates().show()
+------+-----+-------------+                                                    
|  nome|idade|    profissao|
+------+-----+-------------+
| maria|    1|      gerente|
| maria|   37|      gerente|
| paulo|   25|         null|
|tereza|   26|   jornalista|
| pedro|   43|desenvolvedor|
+------+-----+-------------+
```
Se sabemos que a coluna nome é única, então maria parece estar duplicada. O seguinte comando pode ser utilizado para remover linhas duplicadas baseado em colunas específicas.

```python
# Removendo duplicados considerando o match em todas as colunas.
df.dropDuplicates(["nome", "profissao"]).show()
+------+-----+-------------+                                                    
|  nome|idade|    profissao|
+------+-----+-------------+
| pedro|   43|desenvolvedor|
| maria|    1|      gerente|
| paulo|   25|         null|
|tereza|   26|   jornalista|
+------+-----+-------------+
```
Ops, maria possui apenas 1 ano de idade? rsrsrs
O comando manteve apenas o primeiro registro da base. Uma solução seria a seguinte:

```python
# Agrupando dataframe com base nas informações de algumas colunas e retornando o máximo da segunda base.
df = df.groupby(["nome", "profissao"]).max()
# Renomeando coluna do dataframe
df = df.withColumnRenamed("max(idade)", "idade")

df.show()
+------+-------------+----------+
|  nome|    profissao|     idade|
+------+-------------+----------+
| pedro|desenvolvedor|        43|
| maria|      gerente|        37|
| paulo|         null|        25|
|tereza|   jornalista|        26|
+------+-------------+----------+
```

#### Salvando arquivos
Depois dos tratamentos realizados, vamos salvar o arquivo tratado.

```python
df.write.csv('hdfs://localhost/user/files/sample_treated.csv')
```

## Desafio
Faça os tratamentos necessários sobre a base de dados coletada a partir do IBGE.